<a href="https://colab.research.google.com/github/hemanth700/flipkart-major/blob/main/sentiment_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install contractions --quiet
!pip install vaderSentiment --quiet
!pip install emot --quiet

     |████████████████████████████████| 286kB 8.9MB/s 
     |████████████████████████████████| 327kB 38.1MB/s 
     |████████████████████████████████| 133kB 8.1MB/s 


In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

import matplotlib.pyplot as plt
from skimage.io import imread,imshow,imshow_collection

import re
from emot.emo_unicode import UNICODE_EMO, EMOTICONS
import contractions
import nltk
from nltk.tokenize.toktok import ToktokTokenizer
nltk.download('stopwords')
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [3]:
def get_urls(name):
  links=[]
  products=[]
  img_links=[]
  dic={}

  try :
    name=name.replace(' ','%20')
  except:
    pass

  url='https://www.flipkart.com/search?q='+name+'&otracker=search&otracker1=search&marketplace=FLIPKART&as-show=on&as=off'
  print(url)

  page=requests.get(url)
  soup=BeautifulSoup(page.content,"html.parser")

  if soup.findAll("a",attrs={"class":"_1fQZEK"}) != []:
    for link in soup.findAll("a",attrs={"class":"_1fQZEK"}):
      n_link=link.get('href')
      links.append(n_link)
    for link in soup.findAll("img",attrs={"class":"_396cs4 _3exPp9"}):
      n_link=link.get('src')
      img_links.append(n_link)
    for product in soup.findAll("div",attrs={"class":"_4rR01T"}):
      products.append(product.get_text())

  elif soup.findAll("a",attrs={"class":"s1Q9rs"}) != [] :
    for link in soup.findAll("a",attrs={"class":"s1Q9rs"}):
      n_link=link.get('href')
      links.append(n_link)
      products.append(link.get_text())
    for link in soup.findAll("img",attrs={"class":"_396cs4 _3exPp9"}):
      n_link=link.get('src')
      img_links.append(n_link)

  else :
    for link in soup.findAll("a",attrs={"class":"IRpwTa"}):
      n_link=link.get('href')
      links.append(n_link)
    for link in soup.findAll("img",attrs={"class":"_2r_T1I"}):
      n_link=link.get('src')
      img_links.append(n_link)
    for product in soup.findAll("div",attrs={"class":"_2WkVRV"}):
      products.append(product.get_text())

  for i in range(len(links)):
    links[i]="https://www.flipkart.com"+links[i]
    if img_links[i]=='':
      img_links[i]='http://fremontgurdwara.org/wp-content/uploads/2020/06/no-image-icon-2.png'
    dic[products[i]]=[links[i],img_links[i]]
    
  return dic

In [4]:
def get_reviews(new):
  review_link=[]
  reviews=[]
  url=dic[new][0]

  page1=requests.get(url)
  soup=BeautifulSoup(page1.content,"html.parser")
  
  for r in soup.findAll("div",attrs={"class":"t-ZTKy"}):
    reviews.append(r.get_text())
  return reviews

In [5]:
# Remove HTML Tag
def html_tag(text):
  Soup = BeautifulSoup(text,"html.parser")
  new_text = Soup.get_text()
  return new_text

html_tag('<html><h2> some important info </h2></html>')

' some important info '

In [6]:
# Expand Contractions
def con(text):
  expand= contractions.fix(text)
  return expand

con("I don't like you" )

'I do not like you'

In [7]:
#Remove Special Characters
def remove_sp(text):
  pattern = r'[^A-Za-z0-9\s]'
  text = re.sub(pattern,'',text)
  return text

In [8]:
def remove_stopwords(text):

  #creating a stopword list 
  stopword_list = nltk.corpus.stopwords.words('english')
  stopword_list.remove('no')
  stopword_list.remove('not')
  stopword_list.append('read')
  
  len(stopword_list)

  #removing the stop words
  tokenizer = ToktokTokenizer()
  tokens = tokenizer.tokenize(text)
  tokens = [token.strip() for token in tokens]
  filtered_tokens = [token for token in tokens if token not in stopword_list]
  filtered_text = ' '.join(filtered_tokens)
  return filtered_text

In [22]:
def convert_emojis(text):
  for emot in UNICODE_EMO:
    text = text.replace(emot,' '.join(UNICODE_EMO[emot].replace(',','').replace(':','').replace('_',' ').split()))
  return text

In [21]:
#START OF EXECUTION
name=input('Enter the product name : ')
dic=get_urls(name)
#[print(keys, ':', value) for keys,value in dic.items() ]
[print(key) for key in dic.keys()]
new=input("Copy the product name : ")
reviews=get_reviews(new)

# Done Data Acquisation

for i in reviews:
  print(i)
  print()

processed_reviews=reviews.copy()
for i in range(len(reviews)):
  processed_reviews[i]= reviews[i].lower()
  processed_reviews[i]= remove_stopwords(processed_reviews[i])
  processed_reviews[i]= html_tag(processed_reviews[i])
  processed_reviews[i]= con(processed_reviews[i])
  processed_reviews[i]= remove_sp(processed_reviews[i])
  
print('After Preprocessing\n')
for i in processed_reviews :
  print(i)
  print()

# Done Preprocessing

Analyze = SentimentIntensityAnalyzer()
compound1=[]
emoji=[]
for x in range(len(processed_reviews)):
  compound1.append(Analyze.polarity_scores(processed_reviews[x])['compound'])
for x in range(len(reviews)):
  if compound1[x]>=0.8:
    emoji.append('\U0001F600')
  elif compound1[x]<0.8 and compound1[x]>0.1:
    emoji.append('\U0001F642')
  elif compound1[x]<=0.1:
    emoji.append('\U0001F611')

# Done Sentimental Analysis'''

Enter the product name : dress
https://www.flipkart.com/search?q=dress&otracker=search&otracker1=search&marketplace=FLIPKART&as-show=on&as=off
Fashion2wear
Rangun
Tamina
Micozy
dyrectdeals
Selvia
Radius Export/Import
SHEETAL ASSOCIATES
Daevish
METRO-FASHION
Oomph!
FTC FASHIONS
fasense
VSkin
HARPA
Purshottam Wala
yashvi trends
Rudraaksha
Kannan
Adesa
STYLESTONE
Copy the product name : Rudraaksha
5It is really amazing..comfy stuff..very satisfied with this..READ MORE

5nice dress....READ MORE

4Same as shown in picture ☺️READ MORE

After Preprocessing

5it really amazing  comfy stuff  satisfied 

5nice dress 

4same shown picture read



In [24]:
'''
compound score > 0.5 positive
compound score < -0.5 negative
compound score > -0.5 and compound score < 0.5 neutral
'''

'\ncompound score > 0.5 positive\ncompound score < -0.5 negative\ncompound score > -0.5 and compound score < 0.5 neutral\n'

In [25]:
!pip freeze > requirement.txt

In [26]:
!ls

requirement.txt  sample_data
